In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import math

In [2]:
train = pd.read_csv('train.csv')
links = pd.read_csv("data_process/redirected_links.csv",names=["Enlace Real"],  converters={'Enlace Real': eval})
train = train.loc[train['text'].drop_duplicates().index]

In [3]:
train = pd.merge(links, train, right_index=True, left_index=True)

In [4]:
lista = list()
def get_domain(string):
    if len(string)==0:
        return(' ')
    for s in string.split("/"):
        if "." in s:
            return s.replace("www.","")
for index,row in train.loc[train["Enlace Real"].str.len()>0].iterrows():
    for url in row["Enlace Real"]:
        lista.append([index, url, row["target"],get_domain(url)])

In [5]:
enlaces_target = pd.DataFrame(data=lista,columns=["id","Enlace","target","Dominio"])

In [6]:
enlaces_t = enlaces_target.loc[enlaces_target.target == 1]['Dominio'].value_counts()
enlaces_t = pd.DataFrame({'Dominio': enlaces_t.index, 'cant_verdaderos': enlaces_t.values})
enlaces_f = enlaces_target.loc[enlaces_target.target == 0]['Dominio'].value_counts()
enlaces_f = pd.DataFrame({'Dominio': enlaces_f.index, 'cant_falsos': enlaces_f.values})

enlaces = pd.merge(enlaces_t, enlaces_f, how='inner', on='Dominio')
enlaces.head(5)

,Dominio,cant_verdaderos,cant_falsos
0,twitter.com,472,599
1,bbc.co.uk,62,2
2,youtube.com,55,110
3,abcnews.go.com,48,5
4,facebook.com,40,51


In [7]:
enlaces['domain_metric'] = enlaces['cant_verdaderos'] - enlaces['cant_falsos']
enlaces['cant_total'] = enlaces['cant_verdaderos'] + enlaces['cant_falsos']
enlaces['domain_metric'] = enlaces['domain_metric']/enlaces['cant_total']
enlaces = enlaces.sort_values(by='cant_total')
enlaces['cant_total'] = enlaces['cant_total'].apply(lambda x: 1 if x<20 else math.log(x+10,20))
scaler = MinMaxScaler()
x = enlaces['cant_total'].values.reshape(-1, 1)
enlaces['cant_total'] = scaler.fit_transform(x)
enlaces['domain_metric'] = enlaces['domain_metric'] * enlaces['cant_total']

In [8]:
enlaces = enlaces[['Dominio','domain_metric']]
enlaces_target = enlaces_target.merge(enlaces).groupby('id').agg({'domain_metric':['mean','std','max','min']})
enlaces_target.columns =enlaces_target.columns.get_level_values(0) + '_' + enlaces_target.columns.get_level_values(1)
enlaces_target.reset_index(inplace=True)

In [10]:
enlaces_target.to_csv('data_process/enlaces.csv',index=False)

In [11]:
enlaces_target

,id,domain_metric_mean,domain_metric_std,domain_metric_max,domain_metric_min
0,31,-0.118581,NaN,-0.118581,-0.118581
1,32,-0.118581,NaN,-0.118581,-0.118581
2,35,-0.118581,NaN,-0.118581,-0.118581
3,38,-0.181212,NaN,-0.181212,-0.181212
4,44,-0.118581,NaN,-0.118581,-0.118581
...,...,...,...,...,...
2296,7590,-0.049061,NaN,-0.049061,-0.049061
2297,7597,0.000000,NaN,0.000000,0.000000
2298,7601,0.211726,NaN,0.211726,0.211726
2299,7604,-0.181212,NaN,-0.181212,-0.181212
